In [1]:
pip install anchor-exp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 427 kB 8.3 MB/s 
     |████████████████████████████████| 275 kB 55.0 MB/s 
  Created wheel for anchor-exp: filename=anchor_exp-0.0.2.0-py3-none-any.whl size=433521 sha256=1abc5453b05b7b4f217ee2c9578c797287cf2ca8dfc1a2dec2aaf52593ee2002
  Stored in directory: /root/.cache/pip/wheels/ef/70/33/efea781f1c5de2489d9da29b1cfd6053a855423ffbe904a3bc
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=8ec0b79eda2774026a7ef3d3bd1421a9f0b2151ec8ae908a6a48e24e036add52
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built anchor-exp lime


In [2]:
pip install torch transformers spacy && python -m spacy download en_core_web_sm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.1 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 42.2 MB/s 
     |████████████████████████████████| 6.6 MB 35.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 9.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [3]:
%load_ext autoreload
%autoreload 2
import os
import os.path
import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
from sklearn.feature_extraction.text import CountVectorizer
from anchor import anchor_text
import time

In [57]:
import transformers
from transformers import AutoModelForSequenceClassification
# load a BERT sentiment analysis model
tokenizer = transformers.AutoTokenizer.from_pretrained("ProsusAI/finbert")
# model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").cuda()
cl_path = '/content/drive/MyDrive/UNCHANGEDweights/L0/classifier_model/finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3).cuda()


In [5]:
import pandas as pd


In [6]:
df=pd.read_csv("dataforshap.csv", encoding="latin", header=[0])
df

,ï»¿Description,label
0,Finnish Bank of +_land reports its operating p...,negative
1,Finnish Bank of +_land reports operating profi...,negative
2,Finnish electronics contract manufacturer Scan...,negative
3,Finnish electronics contract manufacturer Scan...,negative
4,"Finnish Exel Composites , a technology company...",negative
5,Finnish GeoSentric 's net sales decreased to E...,negative
6,Finnish Kemira 's net sales EUR decreased to E...,negative
7,Finnish plumbing and heating systems supplier ...,negative
8,Finnish plumbing and heating systems supplier ...,negative
9,"Finnish Scanfil , a contract manufacturer and ...",negative


In [7]:
nlp = spacy.load('en_core_web_sm')

In [8]:
import torch
import scipy as sp

In [9]:
explainer = anchor_text.AnchorText(nlp, ['negative', 'positive', 'neutral'], use_unk_distribution=True)

In [58]:
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

In [59]:
def f2(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return int(val)

In [12]:
desc=df['ï»¿Description']

In [13]:
x=desc[3]
print(x)

Finnish electronics contract manufacturer Scanfil reports net sales of EUR 58.9 mn in the second quarter of 2007 , down from EUR 62.4 mn a year earlier .


In [14]:
pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from gensim.parsing.preprocessing import remove_stopwords

y = remove_stopwords(x)

print(y)


Finnish electronics contract manufacturer Scanfil reports net sales EUR 58.9 mn second quarter 2007 , EUR 62.4 mn year earlier .


In [60]:
pred = explainer.class_names[f2([y])]
print(pred)
exp = explainer.explain_instance(y, f, threshold=0.95)
print('Anchor: %s' % (' *** '.join(exp.names())))
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

negative
Anchor: . *** 62.4 *** 58.9 *** manufacturer *** 2007 *** sales *** contract *** Finnish *** earlier *** electronics *** mn *** , *** year *** Scanfil *** net *** second *** EUR *** reports *** EUR *** mn *** quarter
Examples where anchor applies and model predicts negative:



Finnish electronics contract manufacturer Scanfil UNK net UNK UNK UNK mn second UNK 2007 , UNK 62.4 mn UNK e
UNK electronics UNK manufacturer Scanfil reports UNK sales UNK 58.9 mn UNK quarter 2007 UNK UNK 62.4 UNK yea
UNK electronics contract UNK Scanfil reports net UNK EUR 58.9 mn UNK quarter UNK UNK EUR UNK UNK UNK earlier
Finnish UNK contract UNK Scanfil UNK net sales EUR 58.9 UNK UNK UNK 2007 , EUR 62.4 mn year UNK .
UNK electronics contract UNK UNK reports UNK sales UNK UNK UNK second quarter UNK , EUR UNK mn year earlier 
Finnish UNK UNK UNK Scanfil reports net UNK EUR 58.9 mn second quarter UNK UNK EUR UNK mn year UNK .
UNK electronics UNK UNK UNK UNK UNK sales UNK UNK UNK UNK UNK UNK , UNK 62.4 